In [42]:
import numpy as np

# Membership function


def trimf(x, params):
    a, b, c = params
    return np.maximum(np.minimum((x - a) / (b - a), (c - x) / (c - b)), 0)


In [43]:

# Antecedent class
class Antecedent:
    def __init__(self, universe, name):
        self.universe = universe
        self.name = name
        self.membership_functions = {}

    def __setitem__(self, key, params):
        self.membership_functions[key] = params

    def fuzzify(self, value):
        fuzzy_values = {}
        for key, params in self.membership_functions.items():
            fuzzy_values[key] = trimf(value, params)
        return fuzzy_values

# Consequent class


class Consequent:
    def __init__(self, universe, name):
        self.universe = universe
        self.name = name
        self.membership_functions = {}

    def __setitem__(self, key, params):
        self.membership_functions[key] = params

# Rule class


class Rule:
    def __init__(self, antecedents, consequents):
        self.antecedents = antecedents
        self.consequents = consequents

    def evaluate(self, fuzzy_values):
        min_membership = np.inf
        for key, value in self.antecedents.items():
            membership = fuzzy_values[key][value]
            if membership < min_membership:
                min_membership = membership
        consequent_values = {}
        for key, value in self.consequents.items():
            consequent_values[key] = value
        return min_membership, consequent_values

# ControlSystem class


class ControlSystem:
    def __init__(self, rules):
        self.rules = rules

    def evaluate(self, fuzzy_values):
        outputs = {}
        for rule in self.rules:
            membership, consequents = rule.evaluate(fuzzy_values)
            for key, value in consequents.items():
                if key in outputs:
                    outputs[key].append((membership, value))
                else:
                    outputs[key] = [(membership, value)]
        return outputs

# ControlSystemSimulation class


class ControlSystemSimulation:
    def __init__(self, control_system):
        self.control_system = control_system
        self.input = {}
        self.output = {}

    def compute(self):
        fuzzy_values = {}
        for key, value in self.input.items():
            fuzzy_values[key] = self.evaluate_fuzzy_input(key, value)
        outputs = self.control_system.evaluate(fuzzy_values)
        for key, value in outputs.items():
            total_membership = sum([membership for membership, _ in value])
            weighted_sum = sum(
                [membership * output for membership, output in value])
            self.output[key] = weighted_sum / total_membership

    # def evaluate_fuzzy_input(self, variable, value):
    #     return self.control_system[variable].fuzzify(value)

    def evaluate_fuzzy_input(self, variable, value):
        return self.control_system[variable].fuzzify_input(value)

    # def evaluate_fuzzy_input(self, variable, value):
    #     return self.control_system.fuzzify(variable, value)


In [44]:
# Fuzzy variables
stok = Antecedent(np.arange(0, 121, 1), 'stok')
penjualan = Antecedent(np.arange(0, 121, 1), 'penjualan')
pendapatan = Antecedent(np.arange(0, 1700001, 10000), 'pendapatan')
prioritas = Consequent(np.arange(0, 11, 1), 'prioritas')
jumlah = Consequent(np.arange(0, 501, 1), 'jumlah')


In [45]:
# Membership function for stok
stok['sedikit'] = [0, 0, 50]
stok['cukup'] = [30, 60, 90]
stok['banyak'] = [70, 120, 120]

# Membership function for penjualan
penjualan['sedikit'] = [0, 0, 50]
penjualan['sedang'] = [30, 60, 90]
penjualan['banyak'] = [70, 120, 120]

# Membership function for pendapatan
pendapatan['sangat rendah'] = [0, 0, 450000]
pendapatan['rendah'] = [50000, 450000, 850000]
pendapatan['sedang'] = [450000, 850000, 1250000]
pendapatan['tinggi'] = [850000, 1250000, 1650000]
pendapatan['sangat tinggi'] = [1250000, 1700000, 1700000]

# Membership function for prioritas
prioritas['sangat rendah'] = [0, 0, 2]
prioritas['rendah'] = [0, 2, 4]
prioritas['sedang'] = [2, 4, 6]
prioritas['tinggi'] = [4, 6, 8]
prioritas['sangat tinggi'] = [6, 8, 10]

# Membership function for jumlah
jumlah['sedikit'] = [0, 0, 50]
jumlah['sedang'] = [30, 60, 90]
jumlah['banyak'] = [70, 120, 120]


In [46]:
# Rules
rules = [
    Rule({'stok': 'sedikit', 'penjualan': 'sedikit', 'pendapatan': 'sangat rendah'}, {
         'prioritas': 'sangat rendah', 'jumlah': 'sedikit'}),
    Rule({'stok': 'sedikit', 'penjualan': 'sedikit', 'pendapatan': 'rendah'}, {
         'prioritas': 'sangat rendah', 'jumlah': 'sedikit'}),
    Rule({'stok': 'sedikit', 'penjualan': 'sedikit', 'pendapatan': 'rendah'}, {
         'prioritas': 'rendah', 'jumlah': 'sedikit'}),
    Rule({'stok': 'sedikit', 'penjualan': 'sedikit', 'pendapatan': 'tinggi'}, {
         'prioritas': 'rendah', 'jumlah': 'sedikit'}),
    Rule({'stok': 'sedikit', 'penjualan': 'sedikit', 'pendapatan': 'sangat tinggi'}, {
         'prioritas': 'rendah', 'jumlah': 'sedikit'})
]


In [47]:
# Control system
prioritas_jumlah_ctrl = ControlSystem(rules)

# Simulation
prioritas_jumlah = ControlSystemSimulation(prioritas_jumlah_ctrl)


In [48]:
stock = 100
total_penjualan = 20
total_pendapatan = 120000

# Input values
prioritas_jumlah.input['stok'] = stock
prioritas_jumlah.input['penjualan'] = total_penjualan
prioritas_jumlah.input['pendapatan'] = total_pendapatan

# Compute outputs
prioritas_jumlah.compute()

hasil_Prioritas = prioritas_jumlah.output['prioritas']
hasil_jumlah = prioritas_jumlah.output['jumlah']

print("Prioritas:", hasil_Prioritas)
print("Jumlah:", hasil_jumlah)


AttributeError: 'ControlSystem' object has no attribute 'fuzzify'